In [89]:
import json
import pandas as pd
import numpy as np

In [90]:
#  Import the Wikipedia JSON file
# ./Weather_Database/WeatherPy_Database.csv
file_dir = './Resources/'
f'{file_dir}wikipedia-movies.json'

'./Resources/wikipedia-movies.json'

In [91]:
with open(f'{file_dir}/wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file) #now wiki_... a list of dictionary

In [92]:
#Check how many records were pulled in
len(wiki_movies_raw)

7311

In [93]:
# See the first 5 records
wiki_movies_raw[:5]

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
  'year': 1990,
  'imdb_link': 'https://www.imdb.com/title/tt0098987/',
  'title': 'The Adventures of Ford Fairlane',
  'Directed by': 'Renny Harlin',
  'Produced by': ['Steve Perry', 'Joel Silver'],
  'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
  'Story by': ['David Arnott', 'James Cappe'],
  'Based on': ['Characters', 'by Rex Weiner'],
  'Starring': ['Andrew Dice Clay',
   'Wayne Newton',
   'Priscilla Presley',
   'Lauren Holly',
   'Morris Day',
   'Robert Englund',
   "Ed O'Neill"],
  'Narrated by': 'Andrew "Dice" Clay',
  'Music by': ['Cliff Eidelman', 'Yello'],
  'Cinematography': 'Oliver Wood',
  'Edited by': 'Michael Tronick',
  'Productioncompany ': 'Silver Pictures',
  'Distributed by': '20th Century Fox',
  'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
  'Running time': '102 minutes',
  'Country': 'United States',
  'Language': 'English',
  'Budget': '$20 million',


In [94]:
# Some records in the middle
wiki_movies_raw[3600:3605]

[{'url': 'https://en.wikipedia.org/wiki/Benji:_Off_the_Leash!',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0315273/',
  'title': 'Benji: Off the Leash!',
  'Directed by': 'Joe Camp',
  'Written by': 'Joe Camp',
  'Starring': ['Benji', 'Nick Whitaker', 'Shaggy', 'Gypsy the Cockatoo'],
  'Music by': 'Antonio di Lorenzo',
  'Productioncompany ': 'Mulberry Square Productions',
  'Distributed by': 'Mulberry Square Productions',
  'Release date': ['March 26, 2004', '(', '2004-03-26', ')'],
  'Running time': '97 min',
  'Country': 'United States',
  'Language': 'English',
  'Box office': '$3,817,362'},
 {'url': 'https://en.wikipedia.org/wiki/The_Best_Thief_in_the_World',
  'year': 2004,
  'imdb_link': 'https://www.imdb.com/title/tt0389796/',
  'title': 'The Best Thief in the World',
  'Directed by': 'Jacob Kornbluth',
  'Produced by': ['Tim Perrell', 'Nicola Usborne'],
  'Written by': 'Jacob Kornbluth',
  'Starring': ['Marc Rozendaal',
   'Michael Silverman',
   'David Warsh

In [95]:
# Kaggle data is already in flat-file formats, we'll just pull them into Pandas DataFrames 
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')ratings.sample(n=5)

SyntaxError: invalid syntax (<ipython-input-95-a62fffc93555>, line 3)

In [ ]:
# Create a DataFrame from our raw data
wiki_movies_df=pd.DataFrame(wiki_movies_raw)
# wiki_movies_df[3500:3505]
wiki_movies_df.columns.tolist()

In [ ]:
# Use list comprehensions to filter data and save that to an intermediate variable
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie]
len(wiki_movies)

In [ ]:
# Use list comprehensions to filter data and save that to an intermediate variable 
# and get rid of TV shows 
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

## 8.3.5 Create a Function to Clean the Data, Part 1

In [ ]:
# Create a function to clean mouvies with a copy movies and. Any changes we make inside clean_movie() will now only affect the copy, 
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie


In [ ]:
# Filter the list is Arabic
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]

In [ ]:
# More details
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
sort_wiki_movies_df = sorted(wiki_movies_df.columns.tolist())
sort_wiki_movies_df

In [ ]:
# Show non Nan value in the column of the DF
wiki_movies_df["Arabic"].value_counts()

### Handle the Alternative Titles

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
#Step 1: Make an empty dict to hold all of the alternative titles
    alt_titles = {} 
    
# Step 2: Loop through a list of all alternative title keys.
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
# Step 2a: Check if the current key exists in the movie object.
        if key in movie:
# Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary.
            alt_titles[key] = movie[key]
            movie.pop(key)
# Step 3: After looping through every key, add the alternative titles dict to the movie object.
    if len(alt_titles) > 0: #not empty
        movie['alt_titles'] = alt_titles
        
    return movie

In [ ]:
# Make a list of cleaned movies with a list comprehension:
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies) #Set wiki_movies_df to be the DF created from clean_movies
sorted(wiki_movies_df.columns.tolist())

## 8.3.6 Create a Function to Clean the Data, Part 2

In [ ]:
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
#Step 1: Make an empty dict to hold all of the alternative titles
    alt_titles = {} 
    
# Step 2: Loop through a list of all alternative title keys.
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
# Step 2a: Check if the current key exists in the movie object.
        if key in movie:
# Step 2b: If so, remove the key-value pair and add to the alternative titles dictionary.
            alt_titles[key] = movie[key]
            movie.pop(key)
# Step 3: After looping through every key, add the alternative titles dict to the movie object.
    if len(alt_titles) > 0: #not empty
        movie['alt_titles'] = alt_titles
        
    # merge the same column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Directed by', 'Director') #change every instance where the key is "Directed by" to the new key "Director
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
   
    return movie

In [ ]:
# Make a list of cleaned movies with a list comprehension again
clean_movies = [clean_movie(movie) for movie in wiki_movies]
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

## 8.3.7 Remove Duplicate Row

### The columns are tidied up, time to move on to the rows

In [ ]:
# Using str.extract(), which takes in a regular expression pattern. 
# IMDb links finish "(tt\d{7})"
# The parentheses -for one group of text
# The "\d" says to match a numerical digit.
# The "{7}" says to match the last thing (numerical digits) exactly seven times

wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))

# drop any duplicates of IMDb IDs by using the drop_duplicates() method
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

In [ ]:
# Remove Mostly Null Columns
[[column,wiki_movies_df[column].isnull().sum()] for column in wiki_movies_df.columns]

In [96]:
# Make a list of columns that have less than 90% null values 
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id']